#### Google Direction API for Python in ArcPro

In [ ]:
# Install packages
#!pip install pytz

In [ ]:
# import modules
import arcpy, os, sys
from arcpy import env
import pandas as pd
import numpy as np
import requests
import json
import time
import datetime
import pytz
from pytz import timezone

In [ ]:
# set workspace and workfolder
workfolder = r"C:\WORKFOLDER"
workspace = r"C:\WORKSPACE.gdb"

In [ ]:
# request google direction api
# set time
time_str = "1/10/2024 7:15:00 AM"
eastern = timezone('US/Eastern')
time_est = eastern.localize(datetime.datetime.strptime(time_str, "%m/%d/%Y %I:%M:%S %p"))
time_tup = round(time.mktime(time_est.timetuple()))

# set origination
url_o = "https://maps.googleapis.com/maps/api/directions/json?origin="
o_x = 40.875761235775876
o_y = -74.01643973489982
o_coord = str(o_x) + "," + str(o_y)
# set destination
url_d = "&destination="
d_x = 40.7535042462521
d_y = -73.97674839077847
d_coord = str(d_x) + "," + str(d_y)
# set transit mode
url_m = "&mode=transit" #driving
url_md = "&transit_mode=bus|subway|train"
# set preference
url_t = "&departure_time="+str(time_tup)
url_r = "&avoid=tolls|highways|ferries"
# set api key
url_api = "&key=YOURAPIKEY"

# combine url
url_req = url_o + o_coord + url_d + d_coord + url_m + url_md + url_t + url_r + url_api
print(url_req)

# request api
payload={}
headers = {}
response = requests.request("GET", url_req, headers=headers, data=payload)

In [ ]:
# Create empty dataframe with transit mode
tm = ['Total', 'Walking', 'Transit', 'Bicycling', 'Driving']
for x in tm:
     exec('{} = pd.DataFrame()'.format(x))

# Create steps dataframe by different transit mode
routeLen = len(response.json()['routes'][0]['legs'][0]['steps'])
for i in range(routeLen):
    travel_mode = response.json()['routes'][0]['legs'][0]['steps'][i]['travel_mode']
    if travel_mode == 'WALKING':
        Walking = pd.json_normalize(response.json()['routes'][0]['legs'][0]['steps'][i]['steps'])
        Total = pd.concat([Total, Walking])
    elif travel_mode == 'TRANSIT':
        Transit = pd.json_normalize(response.json()['routes'][0]['legs'][0]['steps'][i])
        Total = pd.concat([Total, Transit])
    elif travel_mode == 'BICYCLING':
        Bicycling = pd.json_normalize(response.json()['routes'][0]['legs'][0]['steps'][i])
        Total = pd.concat([Total, Bicycling])
    elif travel_mode == 'DRIVING':
        Driving = pd.json_normalize(response.json()['routes'][0]['legs'][0]['steps'][i])
        Total = pd.concat([Total, Driving])
    else:
        print("There are different transit modes")
        pass


In [ ]:
# Export the dataframe to csv
Total.to_csv(workfolder+'/GoogleDirection.csv')
Total_csv = os.path.join(workfolder, 'GoogleDirection.csv')
print(Total_csv)

In [ ]:
# Convert it to feature class
arcpy.management.XYToLine(
    Total_csv, os.path.join(workspace, 'Line_GoogleDirectionAPI'),
    'start_location.lng', 'start_location.lat',
    'end_location.lng', 'end_location.lat')